#### Example of extracting a Parquet file from a STOQS database and loading it into a Pandas DataFrame for machine learning analysis.

This Notebook is part of the auv-python project (private repository at https://github.com/mbari-org/auv-python). It demonstrates how to read and make interactive plots of millions of data points accessed from a STOQS database.

To execute it (for example):

```bash
    cd GitHub  # Or other appropriate directory on your computer
    git clone https://github.com/mbari-org/auv-python.git
    cd auv-python
    poetry install
    poetry shell
    cd notebooks
    jupyter lab
    # Open this notebook and run it from your browser - interactive zooming does not work in VS Code
```

The url in the cell below was generated by going to https://stoqs.shore.mbari.org/stoqs_mb_diamonds/ and clicking the buttons of the Measured Parameters to be included in the Parquet file. Then clicking the "Measured Parameter Data Access" section and clicking the "Estimate requirements" button to verify that the estimated values are within the available values of the server. The cells below load the data into a Pandas DataFrame and makes interactive zoomable plots.

In [ ]:
import pandas as pd
import panel.widgets as pnw
import pooch

# Construct the url to retrieve the data from the STOQS database:
parms = [
    "ecopuck_bbp700 (m^-1 sr^-1)",
    "ecopuck_cdom (ppb)",
    "ecopuck_chl (ug/l)",
    "hs2_bb420",    # stoqs_mb_diamonds needs reloading with bb*p*420 (m-1)
    "hs2_bb700",    # stoqs_mp_diamonds needs reloading with bb*p*700 (m-1)
    "hs2_fl700",
]
    
stoqs_url = "https://stoqs.shore.mbari.org/stoqs_mb_diamonds/api/measuredparameter.parquet?"
stoqs_url += "parameter__name=" + "&parameter__name=".join(parms)
stoqs_url += "&collect=name&include=activity__name"

# Takes several minutes to retrieve the data the first time, thereafter it's read from a local cache
file_name = pooch.retrieve(
    url = stoqs_url,
    known_hash="63fc3c2df92581981e8a640c74432762ef665c39af6cc989498dfc0dcb9de81b",
    progressbar=True,
)
df = pd.read_parquet(file_name)
df.describe()

In [ ]:
# See that ecopuck data exist at the tail end
df.tail()

In [ ]:
# Use datashader to make interactive biplots of the data
# Do not commit following cell outputs to the repository - they are too big!
import colorcet
import holoviews as hv
from holoviews.operation.datashader import datashade
hv.extension("bokeh")

In [ ]:
# Compare ecopuck and HS2 Chlorophyll
pts_hs2eco = hv.Points(df, ['ecopuck_chl (ug/l)', 'hs2_fl700'])
plots = ( datashade(pts_hs2eco, cmap=colorcet.fire).opts(width=800, height=600) )
plots

In [ ]:
# Compare ecopuck and HS2 420 BB
pts_hs2eco = hv.Points(df, ['ecopuck_bbp700 (m^-1 sr^-1)', 'hs2_bb420'])
plots = ( datashade(pts_hs2eco, cmap=colorcet.fire).opts(width=800, height=600) )
plots

In [ ]:
# Compare ecopuck BB and HS2 700
pts_hs2eco = hv.Points(df, ['ecopuck_bbp700 (m^-1 sr^-1)', 'hs2_bb700'])
plots = ( datashade(pts_hs2eco, cmap=colorcet.fire).opts(width=800, height=600) )
plots

In [ ]:
# Ecopuck BB vs. Chl
pts_hs2eco = hv.Points(df, ['ecopuck_bbp700 (m^-1 sr^-1)', 'ecopuck_chl (ug/l)'])
plots = ( datashade(pts_hs2eco, cmap=colorcet.fire).opts(width=800, height=600) )
plots

In [ ]:
# HS2 BB vs. Chl
pts_hs2eco = hv.Points(df, ['hs2_bb420', 'hs2_fl700'])
plots = ( datashade(pts_hs2eco, cmap=colorcet.fire).opts(width=800, height=600) )
plots

In [ ]:
# Compute proxy_cal_factor for use in 5.2-mpm-bg_biolume-PiO-paper.ipynb and src/data/resample.py
proxy_cal_factor = df['hs2_fl700'].quantile(0.99)
print(f"proxy_cal_factor = {proxy_cal_factor:.6f}")
# proxy_cal_factor = 0.006493